In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import librosa as lr
import librosa.display
import IPython.display as ipd

In [2]:
data_dir = "./Audio"
audio_files = glob(data_dir + "/*.wav")

In [3]:
data = pd.DataFrame()
Emotion = []
Emotional_Intensity = []
Statement = []
Statement_Text = []
Repetition = []
Actor = []

In [4]:
#Creating grand truth target values for training data
for i in range(len(audio_files)):
    temp = audio_files[i][audio_files[i].index("\\")+1:audio_files[0].index(".wav")].split("-")
    
    # creating Emotion
    if temp[2] == "01":
        Emotion.append(1)
    if temp[2] == "02":
        Emotion.append(2)
    if temp[2] == "03":
        Emotion.append(3)
    if temp[2] == "04":
        Emotion.append(4)
    if temp[2] == "05":
        Emotion.append(5)
    if temp[2] == "06":
        Emotion.append(6)
    if temp[2] == "07":
        Emotion.append(7)
    if temp[2] == "08":
        Emotion.append(8)

   ############################

    # creating Emotinal_Intensity
    if temp[3] == "01":
        Emotional_Intensity.append(1)
    if temp[3] == "02":
        Emotional_Intensity.append(2)
        
   #############################

    # creating Statement
    if temp[4] == "01":
        Statement.append(1)
        Statement_Text.append("Kids are talking by the door")
    if temp[4] == "02":
        Statement.append(2)
        Statement_Text.append("Dogs are sitting by the door")
        
    ############################
    
    # creating Repetition
    if temp[5] == "01":
        Repetition.append(1)
    if temp[5] == "02":
        Repetition.append(2)
        
    ###########################
    
    # creating actor gender
    if int(temp[6])%2 == 0:   # even --> female
        Actor.append(1)
    if int(temp[6])%2 == 1:   # odd --> male
        Actor.append(0)

In [5]:
data["Emotion"] = Emotion
data["Emotional_Intensity"] = Emotional_Intensity
data["Statement"] = Statement
data["Statement_Text"] = Statement_Text
data["Repetition"] = Repetition
data["Actor_Gender"] = Actor

In [6]:
#extracting mfcc features from audios
mfcc1 = []
for i in range(0,len(audio_files)):
    
    y, sr = lr.load(audio_files[i],res_type="kaiser_fast",duration=3,offset=0.5)
    sr=np.array(sr)
    mfcc=(lr.feature.mfcc(y=y,hop_length=256,n_mfcc=13))
    #This line controls audios lenght if it is shorter than the trashold algoritm pads mfcc feature with 0
    if (259 > mfcc.shape[1]):
        pad_width = 259 - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        mfcc1.append(mfcc)
    else:
        mfcc1.append(mfcc)
    
mfccTarget1 = pd.DataFrame()
mfccTarget1["mfcc"] = mfcc1

In [7]:
#CNN model with the best accuracies
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten,Dropout,MaxPooling1D,Activation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.asarray(mfccTarget1["mfcc"].tolist()),data["Emotion"].to_numpy(), test_size=0.30, random_state=42,shuffle=True)
model = Sequential()
model.add(Conv1D(256, 5,padding='same', input_shape=(13,259))) #1
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same')) #2
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same')) #3
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same')) #6
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(9)) #7
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100)

Using TensorFlow backend.


Train on 1008 samples, validate on 432 samples
Epoch 1/100
1008/1008 [==============================] - 3s 3ms/step - loss: 20.2733 - accuracy: 0.1359 - val_loss: 6.7728 - val_accuracy: 0.2060
Epoch 2/100
1008/1008 [==============================] - 1s 1ms/step - loss: 3.2882 - accuracy: 0.1925 - val_loss: 2.2677 - val_accuracy: 0.1852
Epoch 3/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.9151 - accuracy: 0.3046 - val_loss: 1.7315 - val_accuracy: 0.3310
Epoch 4/100
1008/1008 [==============================] - 1s 1ms/step - loss: 1.7357 - accuracy: 0.3631 - val_loss: 1.6710 - val_accuracy: 0.3519
Epoch 5/100
1008/1008 [==============================] - 1s 1ms/step - loss: 1.5787 - accuracy: 0.4226 - val_loss: 1.7242 - val_accuracy: 0.3356
Epoch 6/100
1008/1008 [==============================] - 1s 1ms/step - loss: 1.4934 - accuracy: 0.4355 - val_loss: 1.5266 - val_accuracy: 0.4329
Epoch 7/100
1008/1008 [==============================] - 1s 1ms/step - loss: 1.355

1008/1008 [==============================] - 1s 1ms/step - loss: 0.1020 - accuracy: 0.9593 - val_loss: 2.2871 - val_accuracy: 0.5741
Epoch 57/100
1008/1008 [==============================] - 1s 1ms/step - loss: 0.1036 - accuracy: 0.9583 - val_loss: 2.4123 - val_accuracy: 0.5833
Epoch 58/100
1008/1008 [==============================] - 1s 993us/step - loss: 0.1823 - accuracy: 0.9484 - val_loss: 2.1284 - val_accuracy: 0.6157
Epoch 59/100
1008/1008 [==============================] - 1s 971us/step - loss: 0.0697 - accuracy: 0.9772 - val_loss: 2.2766 - val_accuracy: 0.5903
Epoch 60/100
1008/1008 [==============================] - 1s 1ms/step - loss: 0.0493 - accuracy: 0.9831 - val_loss: 2.1896 - val_accuracy: 0.6019
Epoch 61/100
1008/1008 [==============================] - 1s 1ms/step - loss: 0.0636 - accuracy: 0.9762 - val_loss: 2.2320 - val_accuracy: 0.6134
Epoch 62/100
1008/1008 [==============================] - 1s 978us/step - loss: 0.0230 - accuracy: 0.9940 - val_loss: 2.0876 - val_ac